In [1]:
import pandas as pd
import time as t
from datetime import datetime, timedelta
from joblib import Parallel, delayed, parallel_backend
from pandas.tseries.offsets import BDay
import schedule
import numpy as np
import logging
import requests
import json
import pytz
import pickle5 as pickle
from config import *
from numba import njit
import multiprocessing as mp
import concurrent.futures
import os

POLY_URL = 'https://api.polygon.io'
TOKEN = 'sk_17b078529ba34b7396ef93de2d19b287'
BASE_URL = 'https://paper-api.alpaca.markets'
LIVE_URL = 'https://api.alpaca.markets'

LIVE_KEY = 'AKOM736VSX0UQ9UW1KKN'
LIVE_SECRET_KEY = 'A2PFJzveZQF7vVI1f08cWQWTJoXmrmvETBKpNkBH'
PAPER_KEY = 'PKA8BJS8EZJB5B09AGXR'
PAPER_SECRET_KEY = 'cfvyRrOYgvSGfuUsINxwNNHwXXuNnn0MNEbquyw7'

POLY_KEY = 'CPzNjqon5RlL69YIMwM7z3dWpripD4uI'

### Paper or Live
API_URL = BASE_URL
API_KEY = PAPER_KEY
SECRET_KEY = PAPER_SECRET_KEY

ORDERS_URL = "{}/v2/orders".format(API_URL)
ORDERED_URL = "{}/v2/orders?status=filled".format(API_URL)
HEADERS = {'APCA-API-KEY-ID': API_KEY, 'APCA-API-SECRET-KEY': SECRET_KEY}

In [2]:
import itertools

import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime, timedelta
from scipy import stats

from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score, \
                            classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample, class_weight 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

# from imblearn.over_sampling import SMOTE
# from imblearn.combine import SMOTEENN, SMOTETomek

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

In [3]:
holidays = [datetime(2022, 4, 15).date(),
            datetime(2022, 5, 30).date(),
            datetime(2022, 6, 20).date(),
            datetime(2022, 7, 4).date(),
            datetime(2022, 9, 5).date(),
            datetime(2022, 11, 24).date(),
            datetime(2022, 12, 26).date(),
            datetime(2023, 1, 2).date(),
            datetime(2023, 1, 16).date(),
            datetime(2023, 2, 20).date(),
           ]

tzinfo = pytz.timezone("America/New_York")

def load_data(filename):
	with open(f"data/{filename}.pickle", "rb") as f:
		objects = []
		while True:
			try:
				objects.append(pickle.load(f))
			except EOFError:
				break
	return objects[0]

def feature_summary(data):
    print('DataFrame shape')
    print('rows:',data.shape[0])
    print('cols:',data.shape[1])
    col_list = ['Null', '% Null', 'Unique Count', 'Data type', 'Max/Min', 'Mean', 'Std', 'Skewness', 'Sample values']
    df = pd.DataFrame(index=data.columns, columns=col_list)
    df['Null'] = list([len(data[col][data[col].isnull()]) for i, col in enumerate(data.columns)])
    df['% Null'] = list([round(len(data[col][data[col].isnull()]) / data.shape[0] * 100, 2) \
                         for i, col in enumerate(data.columns)])
    df['Unique Count'] = list([len(data[col].unique()) for i, col in enumerate(data.columns)])
    df['Data type'] = list([data[col].dtype for i, col in enumerate(data.columns)])
    for i, col in enumerate(data.columns):
        if 'float' in str(data[col].dtype) or 'int' in str(data[col].dtype):
            df.at[col, 'Max/Min'] = str(round(data[col].max(), 2)) + '/' + str(round(data[col].min(), 2))
            df.at[col, 'Mean'] = data[col].mean()
            df.at[col, 'Std'] = data[col].std()
            df.at[col, 'Skewness'] = data[col].skew()
        df.at[col, 'Sample values'] = list(data[col].unique())
          
    return(df.fillna('-'))

<div class="cite2c-biblio"></div>

Baseline:
1. Volume of last 15 mins >= last 90 days' highest volume * 70%
2. current price >= last 90 day's high * 95%

Others:

3. Volume of last 15 mins >= last 90 days' highest volume * 100%
4. current price >= last 90 day's high * 100%
5. bid_ask_spread_ratio < 1 or same day 0 volume minute bar count >= ? (remove low liquidity stocks)
6. current price > 1 (remove penny stocks)
7. open_price * 1.15 >= current_price > open_price （开盘后涨太多或低于开盘可能动能不足）
8. open_price * 1.25 <= 90 days high （盘前涨太多可能开盘即最高点）
9. (day high - current_price) / (current_price - open_price) <= 0.2 (当日高点回撤太多可能动能不足)
10. current price > 9 trading days ago close （蓄力更久突破更真）
11. if_after_3pm
12. SPY / QQQ / ARKK

<div class="cite2c-biblio"></div>

Signal 1:
1. Volume of last 15 mins >= last 90 days' highest volume * 100%
2. current price >= last 90 day's high * 100%
3. current price > 1 (remove penny stocks)
4. Before 10 am


Signal 2:
1. Volume of last 15 mins >= last 90 days' highest volume * 100%
2. current_price > open_price * 1.15 (追高)
3. current price > 1 (remove penny stocks)
4. Before 12 am

#### Back Test

In [134]:
def check_zero_vol_mins_count(today_df, idx, open_time):
    prev = today_df.iloc[:idx]
    return (today_df.t[idx] - open_time).total_seconds() / 60.0 - len(prev)
    
def get_other_conditions(today_df, idx, v, max_vol, max_high, open_time):
    open_price = today_df.loc[0, 'o']
    curr = today_df.h[idx]
    
    price_max_ratio = round(100 * curr / max_high, 2)
    vol_max_ratio = round(100 * v / max_vol, 2)
    zero_vol_mins_count = check_zero_vol_mins_count(today_df, idx, open_time)
    
    if_curr_higher_one = int(curr > 1)
    
    if_curr_higher_open = int(curr >= open_price)
    if_curr_not_too_high = int(curr <= open_price * 1.15)
    if_curr_not_too_high_90_days = int(curr <= max_high * 1.25)
    
    return [price_max_ratio, vol_max_ratio, zero_vol_mins_count, if_curr_higher_one,
            if_curr_higher_open, if_curr_not_too_high, if_curr_not_too_high_90_days]

def strategy(today_df, max_vol, max_high, vol_=1, high_=1):
    # Trading hours only
    close_time = pd.DatetimeIndex(today_df.t)[0].replace(hour=15, minute=59, second=0, microsecond=0)
    open_time = pd.DatetimeIndex(today_df.t)[0].replace(hour=9, minute=30, second=0, microsecond=0)
    
    today_df['t'] = pd.DatetimeIndex(today_df.t)
    today_df = today_df[(today_df.t >= open_time) & (today_df.t <= close_time)].reset_index().drop(columns='index')
    
    for idx, v in enumerate(today_df.v_15m):
        if not baseline_conditions(current_v_15m=v, 
                                   max_vol=max_vol, 
                                   current_price=today_df.h[idx], 
                                   max_high=max_high, 
                                   vol_=vol_, 
                                   high_=high_):
            continue
        
        other_condi = get_other_conditions(today_df, idx, v, max_vol, max_high, open_time)
        buy_price = today_df.c[idx]
        entry_time = today_df.t[idx]
        
        # Find exit
        monitor_df = today_df[today_df.t > entry_time].reset_index().drop(columns='index')
        # Standard exit
        for i in range(len(monitor_df)):
            if monitor_df.l[i] < buy_price * 0.99 and monitor_df.h[i] > buy_price * 1.01:
                res = [np.nan, entry_time, buy_price, monitor_df.t[i], np.nan]
                res.extend(other_condi)
                return res
            
            if monitor_df.l[i] < buy_price * 0.99:
                res = [-1, entry_time, buy_price, monitor_df.t[i], buy_price * 0.99]
                res.extend(other_condi)
                return res
            
            if monitor_df.h[i] > buy_price * 1.01:
                monitor_df2 = today_df[today_df.t > monitor_df.t[i]].reset_index().drop(columns='index')
                res = stra_helper(monitor_df2, buy_price, entry_time, close_time)
                res.extend(other_condi)
                return res
            
            if monitor_df.t[i] >= close_time:
                res = [100 * (monitor_df.c[i] - buy_price) / buy_price, 
                       entry_time, buy_price, monitor_df.t[i], monitor_df.h[i]]
                res.extend(other_condi)
                return res
            
            continue
            
def stra_helper(monitor_df2, buy_price, entry_time, close_time):
    for i in range(len(monitor_df2)):
        if monitor_df2.h[i] >= buy_price * 1.03:
            return [3, entry_time, buy_price, monitor_df2.t[i], buy_price * 1.03]
        if monitor_df2.l[i] < buy_price:
            return [0, entry_time, buy_price, monitor_df2.t[i], buy_price]
        if monitor_df2.t[i] >= close_time:
            res = 100 * (monitor_df2.c[i] - buy_price) / buy_price
            return [res, entry_time, buy_price, monitor_df2.t[i], monitor_df2.h[i]]
        
        continue

@njit
def njit_baseline_conditions(current_v_15m, max_vol, current_price, max_high, vol_=1, high_=1):
    if current_v_15m >= vol_ * max_vol and current_price >= high_ * max_high:
        return True
    return False

def baseline_conditions(current_v_15m, max_vol, current_price, max_high, vol_=1, high_=1):
    if current_v_15m >= vol_ * max_vol and current_price >= high_ * max_high:
        return True
    return False

def get_moving_15m_max_volume(ticker_1_min_df):

    v_15m = [np.nan]
    v = np.array(ticker_1_min_df.v)
    rows = ticker_1_min_df.shape[0]
    for idx in range(1, rows):
        if idx < 15:
            v_15m.append(v[:idx].sum())
        else:
            v_15m.append(v[idx - 15: idx].sum())
    return v_15m

def get_today_data(ticker, date):
    response = requests.get(f'{POLY_URL}/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?sort=asc&apiKey={POLY_KEY}')
    res = json.loads(response.content)['results']
    today_df = pd.DataFrame(res)

    today_df['t'] = [datetime.fromtimestamp(t / 1000, 
                    tzinfo).strftime('%Y-%m-%d %H:%M:%S') \
                    for t in today_df['t']]
    today_df['v_15m'] = get_moving_15m_max_volume(today_df)
    return today_df

def backtest(saved_data, start_date, end_date, vol_=1, high_=1):
    failed_ticker = []

    results = []
    for ticker in saved_data.keys():
        # Every minute - check previous 15 min vol sum & high check
        saved_data_ticker = pd.DataFrame(saved_data[ticker])
        saved_data_ticker['datetime'] = pd.DatetimeIndex(saved_data_ticker.time).date


        days_delta = (end_date - start_date).days

        for i in range(days_delta + 1):
            date = start_date + timedelta(days=i)

            # Skip if not trading day
            if date.weekday() >= 5 or date in holidays:
                continue

            # Get past 90 days data
            past_90_df = saved_data_ticker[(saved_data_ticker.datetime >= date - timedelta(days=90)) &
                                          (saved_data_ticker.datetime < date)]

            # Get past 90 days max_vol and max_high
            max_vol = past_90_df.volume.max()
            max_high = past_90_df.high.max()

            try:
                # Get today data
                today_df = get_today_data(ticker, date)
                # If condition fits
                if not baseline_conditions(current_v_15m=today_df['v_15m'].max(), 
                                       max_vol=max_vol, 
                                       current_price=today_df['h'].max(), 
                                       max_high=max_high, 
                                       vol_=vol_, 
                                       high_=high_):
                    continue

                res = strategy(today_df, max_vol, max_high, vol_=vol_, high_=high_)
                if res:
                    res.append(ticker)
                    results.append(res)
                    print(res)

            except: 
                failed_ticker.append(ticker)
                pass

    return results, failed_ticker

def parallel_ticker_backtest(args):
    return parallel_backtest(*args)

def parallel_backtest(ticker, saved_data, start_date, end_date, vol_=1, high_=1):
    failed_ticker = []

    results = []
    # Every minute - check previous 15 min vol sum & high check
    saved_data_ticker = pd.DataFrame(saved_data[ticker])
    saved_data_ticker['datetime'] = pd.DatetimeIndex(saved_data_ticker.time).date

    days_delta = (end_date - start_date).days

    for i in range(days_delta + 1):
        date = start_date + timedelta(days=i)

        # Skip if not trading day
        if date.weekday() >= 5 or date in holidays:
            continue

        # Get past 90 days data
        past_90_df = saved_data_ticker[(saved_data_ticker.datetime >= date - timedelta(days=90)) &
                                      (saved_data_ticker.datetime < date)]

        # Get past 90 days max_vol and max_high
        max_vol = past_90_df.volume.max()
        max_high = past_90_df.high.max()

        try:
            # Get today data
            today_df = get_today_data(ticker, date)
            # If condition fits
            if not njit_baseline_conditions(current_v_15m=today_df['v_15m'].max(), 
                                       max_vol=max_vol, 
                                       current_price=today_df['h'].max(), 
                                       max_high=max_high, 
                                       vol_=vol_, 
                                       high_=high_):
                continue

            res = strategy(today_df, max_vol, max_high, vol_=vol_, high_=high_)
            if res:
                res.append(ticker)
                results.append(res)
                print(res)

        except: 
            failed_ticker.append(ticker)
            pass

    return results, failed_ticker

if __name__ == '__main__':
    
    saved_data = load_data('data')
    start_date = datetime(2022, 4, 1).date()
    end_date = datetime(2023, 3, 31).date()

    tickers = list(saved_data.keys())
    num_threads = min(len(tickers), (os.cpu_count() or 1) * 5)  # Adjust the number of threads based on your requirements

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        arguments = [(ticker, saved_data, start_date, end_date, 0.7, 0.95) for ticker in tickers]
        results_and_failed = list(executor.map(parallel_ticker_backtest, arguments))

    results = [res for res_list, _ in results_and_failed for res in res_list if res is not None]
    failed_ticker = [failed for _, failed_list in results_and_failed for failed in failed_list]

[-1, Timestamp('2022-04-01 09:59:00'), 3.5282, Timestamp('2022-04-01 13:26:00'), 3.492918, 95.88, 421.04, 22.0, 1, 0, 1, 1, 'AAIC']
[-1, Timestamp('2022-04-07 12:58:00'), 3.5219, Timestamp('2022-04-07 12:59:00'), 3.486681, 102.08, 82.59, 199.0, 1, 1, 1, 1, 'AAME']
[-1, Timestamp('2022-04-11 15:20:00'), 3.58, Timestamp('2022-04-11 15:28:00'), 3.5442, 100.28, 90.79, 334.0, 1, 1, 1, 1, 'AAME']
[-1, Timestamp('2022-04-20 13:05:00'), 2.57, Timestamp('2022-04-20 13:06:00'), 2.5443, 99.23, 70.4, 174.0, 1, 1, 1, 1, 'ABIO']
[0.026838432635547818, Timestamp('2022-04-27 15:58:00'), 37.26, Timestamp('2022-04-27 15:59:00'), 37.32, 95.35, 73.31, 122.0, 1, 1, 1, 1, 'AAT']
[-1, Timestamp('2022-04-21 09:40:00'), 21.21, Timestamp('2022-04-21 09:49:00'), 20.9979, 100.62, 72.58, 0.0, 1, 0, 1, 1, 'AAL']
[-0.09380863039399424, Timestamp('2022-06-24 15:55:00'), 10.66, Timestamp('2022-06-24 15:59:00'), 10.65, 98.8, 76.18, 0.0, 1, 1, 1, 1, 'ABCL']
[-1, Timestamp('2022-07-21 10:49:00'), 1.53, Timestamp('2022-07

In [135]:
df_res = pd.DataFrame(results, columns=['return', 'entry_time', 'entry_price', 'exit_time', 
                               'exit_price', 'price_max_ratio', 'vol_max_ratio', 'zero_vol_mins_count', 
                               'if_curr_higher_one', 'if_curr_higher_open', 
                               'if_curr_not_too_high', 'if_curr_not_too_high_90_days',
                               'ticker'])

In [136]:
df_res.to_csv('data/signals_backtest_raw.csv', index=False)

##### Basic processing

In [49]:
df_res = pd.read_csv('data/signals_backtest_raw.csv')
df_res['entry_time'] = pd.DatetimeIndex(df_res['entry_time'])
df_res['exit_time'] = pd.DatetimeIndex(df_res['exit_time'])

df_res['date'] = df_res['entry_time'].dt.date
df_res['entry_hour'] = df_res['entry_time'].dt.hour
df_res['if_entry_after_3pm'] = (df_res['entry_hour'] >= 15).astype(int)

df_res['entry_time'] = df_res['entry_time'].dt.time
df_res['exit_time'] = df_res['exit_time'].dt.time

df_res['date'] = pd.DatetimeIndex(df_res['date'])
df_res['month'] = df_res['date'].dt.to_period('M')
df_res['weekday'] = df_res['date'].dt.weekday
df_res['if_entry_before_10am'] = (df_res['entry_hour'] == 9).astype(int)

df_res['if_price_max'] = (df_res['price_max_ratio'] >= 100).astype(int)
df_res['if_vol_max'] = (df_res['vol_max_ratio'] >= 100).astype(int)
df_res['if_vol_count_zero'] = (df_res['zero_vol_mins_count'] == 0).astype(int)

df_res = df_res[['date', 'month', 'weekday', 'entry_hour', 
                 'return', 'ticker', 
                 'entry_time', 'entry_price', 'exit_time', 'exit_price',
                 'price_max_ratio', 'vol_max_ratio', 'if_price_max', 'if_vol_max', 
                 'zero_vol_mins_count', 'if_vol_count_zero',
                 'if_curr_higher_one', 'if_curr_higher_open', 'if_curr_not_too_high',
                 'if_curr_not_too_high_90_days', 
                 'if_entry_after_3pm', 'if_entry_before_10am'
                ]]

df_res = df_res.sort_values(by='date')

# df_res.to_csv('data/signals_backtest.csv', index=False)

#### EDA

In [289]:
df_res = pd.read_csv('data/signals_backtest.csv')

In [290]:
temp = df_res.copy()

print('All Signals')
print('')
print('Avg return per trade: ', round(temp['return'].mean(), 3))
print('Count: ', temp.shape[0])

All Signals

Avg return per trade:  0.04
Count:  7945


In [291]:
df_res.groupby('month')['return'].sum()

month
2022-04    -38.876972
2022-05     -3.505966
2022-06      1.513059
2022-07     52.970488
2022-08    133.685206
2022-09     13.736742
2022-10     38.967880
2022-11     12.803880
2022-12     11.176999
2023-01     64.523862
2023-02     33.107399
2023-03    -14.210309
Name: return, dtype: float64

In [292]:
df_res.groupby('entry_hour')['return'].sum()

entry_hour
9     242.658707
10     28.834470
11     61.819286
12     -4.806086
13    -13.369714
14      3.162423
15    -12.406817
Name: return, dtype: float64

In [293]:
temp1 = df_res[df_res['price_max_ratio'] >= \
       100][df_res['vol_max_ratio'] >= \
            100][df_res['if_curr_higher_one'] == \
                 1][df_res['entry_hour'] == 9].dropna(subset=['return'])

print('Avg return per trade: ', round(temp1['return'].mean(), 3))
print('Count: ', temp1.shape[0])

Avg return per trade:  0.443
Count:  148


In [294]:
temp2 = df_res[df_res['vol_max_ratio'] >= \
            100][df_res['if_curr_not_too_high'] == \
                 0][df_res['if_curr_higher_one'] == \
                 1][df_res['entry_hour'] <= 11].dropna(subset=['return'])

print('Avg return per trade: ', round(temp2['return'].mean(), 3))
print('Count: ', temp2.shape[0])

Avg return per trade:  0.826
Count:  69


In [295]:
idx1 = temp1.index.tolist()
idx2 = temp2.index.tolist()
temp3 = df_res[(df_res.index.isin(idx1)) | (df_res.index.isin(idx2))]

print('Avg return per trade: ', round(temp3['return'].mean(), 3))
print('Count: ', temp3.shape[0])

Avg return per trade:  0.51
Count:  209


In [296]:
df_res[df_res['price_max_ratio'] >= \
       100][df_res['vol_max_ratio'] >= \
            100][df_res['if_curr_higher_one'] == \
                 1].groupby(['entry_hour'])['return'].agg({'mean', 'sum', 'count'})

,mean,sum,count
entry_hour,,,
9,0.442979,65.560826,148
10,0.262099,9.959747,38
11,-0.026805,-0.804161,30
12,0.215977,5.399434,25
13,0.148728,5.800398,39
14,0.221935,5.326441,24
15,0.057896,2.431639,42


##### Grid Search

In [144]:
if_price_max_list = [[1], [0], [1, 0]]
if_vol_max_list = [[1], [0], [1, 0]]
if_vol_count_zero_list = [[1], [0], [1, 0]]
if_curr_higher_one_list = [[1], [0], [1, 0]]
if_curr_higher_open_list = [[1], [0], [1, 0]]
if_curr_not_too_high_list = [[1], [0], [1, 0]]
if_curr_not_too_high_90_days_list = [[1], [0], [1, 0]]
entry_hour_list = [9, 10, 11, 12, 13, 14, 15]

from itertools import product
param_set = list(product(if_price_max_list,
                        if_vol_max_list,
                        if_vol_count_zero_list,
                        if_curr_higher_one_list,
                        if_curr_higher_open_list,
                        if_curr_not_too_high_list,
                        if_curr_not_too_high_90_days_list,
                        entry_hour_list))

grid_search_res = []
for i in param_set:
    filter_df = df_res[(df_res.if_price_max.isin(i[0])) &
                   (df_res.if_vol_max.isin(i[1])) &
                   (df_res.if_vol_count_zero.isin(i[2])) &
                   (df_res.if_curr_higher_one.isin(i[3])) &
                   (df_res.if_curr_higher_open.isin(i[4])) &
                   (df_res.if_curr_not_too_high.isin(i[5])) &
                   (df_res.if_curr_not_too_high_90_days.isin(i[6])) &
                   (df_res.entry_hour <= i[7])
                  ]
    res = [filter_df['return'].mean(), filter_df.shape[0]]
    res.extend(i)
    grid_search_res.append(res)

In [145]:
grid_sear_df = pd.DataFrame(grid_search_res, 
                            columns=['return', 
                                     'count', 
                                     'if_price_max',
                                     'if_vol_max',
                                     'if_vol_count_zero',
                                     'if_curr_higher_one',
                                     'if_curr_higher_open',
                                     'if_curr_not_too_high',
                                     'if_curr_not_too_high_90_days',
                                     'entry_hour'])

In [182]:
grid_sear_df[
        (grid_sear_df['count'] > 200)  
#         & (grid_sear_df['entry_hour'] == 9)
        ][grid_sear_df['if_curr_not_too_high'].apply(lambda x: x[0] == 0)].sort_values(by='return', ascending=False).head(30)

,return,count,if_price_max,if_vol_max,if_vol_count_zero,if_curr_higher_one,if_curr_higher_open,if_curr_not_too_high,if_curr_not_too_high_90_days,entry_hour
11366,0.550847,207,"[1, 0]",[1],"[1, 0]",[1],[1],[0],[1],14
11492,0.550847,207,"[1, 0]",[1],"[1, 0]",[1],"[1, 0]",[0],[1],14
11380,0.520325,218,"[1, 0]",[1],"[1, 0]",[1],[1],[0],"[1, 0]",14
11506,0.520325,218,"[1, 0]",[1],"[1, 0]",[1],"[1, 0]",[0],"[1, 0]",14
14765,0.514706,300,"[1, 0]","[1, 0]","[1, 0]",[1],[1],[0],[1],11
14891,0.514706,300,"[1, 0]","[1, 0]","[1, 0]",[1],"[1, 0]",[0],[1],11
11367,0.506986,231,"[1, 0]",[1],"[1, 0]",[1],[1],[0],[1],15
11493,0.506986,231,"[1, 0]",[1],"[1, 0]",[1],"[1, 0]",[0],[1],15
14779,0.502242,337,"[1, 0]","[1, 0]","[1, 0]",[1],[1],[0],"[1, 0]",11
14905,0.502242,337,"[1, 0]","[1, 0]","[1, 0]",[1],"[1, 0]",[0],"[1, 0]",11


In [73]:
feature_summary(grid_sear_df)

DataFrame shape
rows: 896
cols: 10


,Null,% Null,Unique Count,Data type,Max/Min,Mean,Std,Skewness,Sample values
return,671,74.89,135,float64,3.0/-1.0,0.090651,0.972723,1.503730,"[0.20117417522874256, -0.7436795911934047, 0.1..."
count,0,0.00,71,int64,644/0,8.867188,44.034106,8.686487,"[107, 5, 1, 2, 0, 58, 6, 3, 7, 15, 23, 26, 21,..."
if_price_max_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
if_vol_max_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
if_vol_count_zero_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
if_curr_higher_one_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
if_curr_higher_open_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
if_curr_not_too_high_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
if_curr_not_too_high_90_days_list,0,0.00,2,int64,1/0,0.500000,0.500279,0.000000,"[1, 0]"
entry_hour_list,0,0.00,7,int64,15/9,12.000000,2.001117,0.000000,"[9, 10, 11, 12, 13, 14, 15]"


In [15]:
t = df_res[df_res['price_max_ratio'] >= \
       100][df_res['vol_max_ratio'] >= \
            100][df_res['if_curr_higher_one'] == \
                 1][df_res['if_curr_higher_open'] == \
                    1][df_res['entry_hour'] == 9]

t['return'].mean()

0.43257732507878155

In [16]:
df_res[~df_res.index.isin(t.index)]['return'].mean()

0.03306353684411205

In [166]:
t.groupby('month')['return'].sum()

month
2022-04    -2.000000
2022-05     0.000000
2022-06     3.000000
2022-07     1.099108
2022-08    15.415205
2022-09     3.000000
2022-10     1.000000
2022-11     0.100000
2022-12     7.886051
2023-01     2.761991
2023-02    13.143200
2023-03    15.155270
Freq: M, Name: return, dtype: float64

In [167]:
df_res.groupby('month')['return'].sum()

month
2022-04    -38.876972
2022-05     -3.505966
2022-06      1.513059
2022-07     52.970488
2022-08    133.685206
2022-09     13.736742
2022-10     38.967880
2022-11     12.803880
2022-12     11.176999
2023-01     64.523862
2023-02     33.107399
2023-03    -14.210309
Freq: M, Name: return, dtype: float64

In [168]:
df_res['return'].mean()

0.040461940329939335

In [169]:
df_res[df_res['month'] == '2023-3']

,date,return,ticker,entry_time,entry_price,exit_time,exit_price,price_max_ratio,vol_max_ratio,zero_vol_mins_count,if_curr_higher_one,if_curr_higher_open,if_curr_not_too_high,if_curr_not_too_high_90_days,if_entry_after_3pm,entry_hour,month
2977,2023-03-01,-1.000000,GDRX,09:52:00,5.9300,09:53:00,5.870700,95.20,73.46,0.0,1,0,1,1,0,9,2023-03
6411,2023-03-01,0.120274,SITM,15:56:00,133.0300,15:59:00,133.190000,98.76,75.69,103.0,1,1,1,1,1,15,2023-03
1525,2023-03-01,-1.000000,CHCI,10:28:00,6.2850,10:57:00,6.222150,97.14,74.56,29.0,1,0,1,1,0,10,2023-03
3551,2023-03-01,0.391656,IBP,15:56:00,117.4500,15:59:00,117.920000,99.37,70.43,154.0,1,1,1,1,1,15,2023-03
2306,2023-03-01,-1.000000,DY,09:40:00,103.4150,09:41:00,102.380850,102.42,75.48,1.0,1,1,1,1,0,9,2023-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,2023-03-31,-1.000000,ASM,13:01:00,0.9099,13:06:00,0.900801,102.01,71.21,76.0,0,1,1,1,0,13,2023-03
6559,2023-03-31,0.000000,SONM,10:39:00,0.9400,10:49:00,0.940000,96.94,96.22,36.0,0,1,1,1,0,10,2023-03
5250,2023-03-31,1.119910,OMAB,11:43:00,88.4000,15:59:00,89.460000,98.63,85.51,85.0,1,1,1,1,0,11,2023-03
3631,2023-03-31,0.000000,IMMR,09:35:00,8.8200,09:57:00,8.820000,104.87,88.32,0.0,1,1,1,1,0,9,2023-03


In [175]:
df_res.shape

(7945, 17)

#### ML

In [69]:
df_ml = pd.read_csv('data/signals_backtest.csv')

df_ml['weekday'] = pd.DatetimeIndex(df_ml.date).weekday
df_ml['if_entry_before_10am'] = (df_ml['entry_hour'] == 9).astype(int)
df_ml['target'] = (df_ml['return'] == 3).astype(int)

df_ml['price_max_ratio'] = df_ml['price_max_ratio'] - 95
df_ml['vol_max_ratio'] = df_ml['vol_max_ratio'] - 70

df_ml['if_price_max'] = (df_ml['price_max_ratio'] >= 5).astype(int)
df_ml['if_vol_max'] = (df_ml['vol_max_ratio'] >= 30).astype(int)

df_ml = df_ml[['entry_price', 'price_max_ratio', 'vol_max_ratio', 'if_price_max',
              'if_vol_max', 'zero_vol_mins_count', 'if_curr_higher_one', 
              'if_curr_higher_open', 'if_curr_not_too_high',
              'if_curr_not_too_high_90_days', 'if_entry_after_3pm', 'entry_hour', 'if_entry_before_10am',
              'weekday', 'target']]

In [70]:
df_train = df_ml.iloc[:6300]
df_test = df_ml.iloc[6300:]

In [22]:
def get_col_avg_dict(data, col, target='target'):
    unique_item = data[col].unique()
    dic = {item: data[data[target] == 1][data[col] == item].shape[0] * 100 / data[data[col] == item].shape[0]
           for item in unique_item}
    return dic

def target_encoder(avg_cols, X_train, X_test, y_train):
    for col in avg_cols:
        res = get_col_avg_dict(pd.concat([X_train, y_train], axis=1), col, target='target')
        X_train[col] = X_train[col].map(res)
        X_test[col] = X_test[col].map(res)
    return X_train, X_test

def get_feature_importance(X_train, model):
    cols = X_train.columns.tolist()
    feature_importance = pd.DataFrame([cols, model.feature_importances_]).T
    feature_importance.columns = ['Features', 'Scores']
    feature_importance = feature_importance.sort_values(by='Scores', ascending=False)
    return feature_importance

def xgb_predict_test(training_fixed, test_fixed, selected_cols,
                      target_encode_cols=[], 
                      max_depth=5, learning_rate=0.4, eval_metric='logloss', 
                      proba_thresh=0.5, subsample=1, scale_pos_weight=10, 
                      min_child_weight=10, colsample_bytree=0.7):
    
    X_train = training_fixed[selected_cols]
    y_train = training_fixed.target
    X_test = test_fixed[selected_cols]
    y_test = test_fixed.target
    
    print(f'training data size: {len(X_train)} - target: {round(y_train.mean() * 100, 2)}%')
    print(f'testing data size: {len(X_test)} - target: {round(y_test.mean() * 100, 2)}%')
    print('')
    
    # Target Encoding by training only
    X_train, X_test = target_encoder(target_encode_cols, X_train, X_test, y_train)
        
    # Model Fit
    model = XGBClassifier(objective='binary:logistic', 
                          scale_pos_weight=scale_pos_weight, 
                          max_depth=max_depth,
                          learning_rate=learning_rate,
                          subsample=subsample,
                          min_child_weight=min_child_weight, 
                          colsample_bytree=colsample_bytree)
    model.fit(X_train, y_train, eval_metric=eval_metric)

    # Make predictions for test data
    y_pred = (model.predict_proba(X_test)[:,1] >= proba_thresh).astype(bool)
    y_pred_prob = model.predict_proba(X_test) 
    
    print(f'Accuracy: {round(accuracy_score(y_test, y_pred) * 100, 2)}%')
    print(f'Precision: {round(precision_score(y_test, y_pred) * 100, 2)}%')
    print(f'Recall: {round(recall_score(y_test, y_pred) * 100, 2)}%')
    print(f'F1: {round(f1_score(y_test, y_pred) * 100, 2)}%')
    print(f'ROC_AUC: {round(roc_auc_score(y_test, y_pred) * 100, 2)}%')
    print(f'Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    
    # Get Feature Importance
    feature_importance = get_feature_importance(X_train, model)
    return feature_importance, y_pred

In [23]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

def xgb_predict_test_kfold(training_fixed, test_fixed, selected_cols,
                           target_encode_cols=[], 
                           max_depth=5, learning_rate=0.4, eval_metric='logloss', 
                           proba_thresh=0.5, subsample=1, scale_pos_weight=10, 
                           min_child_weight=10, colsample_bytree=0.7, n_splits=5):
    
    X = training_fixed[selected_cols]
    y = training_fixed.target
    X_test = test_fixed[selected_cols]
    y_test = test_fixed.target
    
    print(f'training data size: {len(X)} - target: {round(y.mean() * 100, 2)}%')
    print(f'testing data size: {len(X_test)} - target: {round(y_test.mean() * 100, 2)}%')
    print('')

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    acc_scores, prec_scores, rec_scores, f1_scores, roc_auc_scores = [], [], [], [], []
    
    for train_index, val_index in kfold.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Target Encoding by training only
        X_train, X_val = target_encoder(target_encode_cols, X_train, X_val, y_train)
        
        # Model Fit
        model = XGBClassifier(objective='binary:logistic', 
                              scale_pos_weight=scale_pos_weight, 
                              max_depth=max_depth,
                              learning_rate=learning_rate,
                              subsample=subsample,
                              min_child_weight=min_child_weight, 
                              colsample_bytree=colsample_bytree)
        model.fit(X_train, y_train, eval_metric=eval_metric)

        # Make predictions for validation data
        y_pred = (model.predict_proba(X_val)[:,1] >= proba_thresh).astype(bool)
        y_pred_prob = model.predict_proba(X_val) 

        # Append scores for the current fold
        acc_scores.append(accuracy_score(y_val, y_pred))
        prec_scores.append(precision_score(y_val, y_pred))
        rec_scores.append(recall_score(y_val, y_pred))
        f1_scores.append(f1_score(y_val, y_pred))
        roc_auc_scores.append(roc_auc_score(y_val, y_pred))

    print(f'Average Accuracy: {round(np.mean(acc_scores) * 100, 2)}%')
    print(f'Average Precision: {round(np.mean(prec_scores) * 100, 2)}%')
    print(f'Average Recall: {round(np.mean(rec_scores) * 100, 2)}%')
    print(f'Average F1: {round(np.mean(f1_scores) * 100, 2)}%')
    print(f'Average ROC_AUC: {round(np.mean(roc_auc_scores) * 100, 2)}%')

    # Get Feature Importance
    feature_importance = get_feature_importance(X_train, model)
    return feature_importance


In [24]:
from sklearn.model_selection import GridSearchCV

def xgb_predict_test_gridsearch(training_fixed, test_fixed, selected_cols,
                                target_encode_cols=[], 
                                max_depth=5, learning_rate=0.4, eval_metric='logloss', 
                                proba_thresh=0.5, subsample=1, scale_pos_weight=10, 
                                min_child_weight=10, colsample_bytree=0.7, n_splits=5,
                                param_grid=None):
    
    if param_grid is None:
        param_grid = {
            'max_depth': [3, 5, 7],
            'learning_rate': [0.1, 0.2, 0.7],
            'subsample': [0.7, 1],
            'min_child_weight': [1, 10],
            'colsample_bytree': [0.7, 1]
        }
    
    X = training_fixed[selected_cols]
    y = training_fixed.target
    X_test = test_fixed[selected_cols]
    y_test = test_fixed.target
    
    # print(f'training data size: {len(X)} - target: {round(y.mean() * 100, 2)}%')
    # print(f'testing data size: {len(X_test)} - target: {round(y_test.mean() * 100, 2)}%')
    # print('')

    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    model = XGBClassifier(objective='binary:logistic', 
                          scale_pos_weight=scale_pos_weight, 
                          max_depth=max_depth,
                          learning_rate=learning_rate,
                          subsample=subsample,
                          min_child_weight=min_child_weight, 
                          colsample_bytree=colsample_bytree)

    grid_search = GridSearchCV(model, param_grid, cv=kfold, scoring='f1', verbose=2, n_jobs=-1)
    grid_search.fit(X, y)

    print("Best parameters found: ", grid_search.best_params_)
    print("Best score found: ", grid_search.best_score_)

    best_model = grid_search.best_estimator_

    # Make predictions for test data
    y_pred = (best_model.predict_proba(X_test)[:,1] >= proba_thresh).astype(bool)
    y_pred_prob = best_model.predict_proba(X_test) 

    # print(f'Accuracy: {round(accuracy_score(y_test, y_pred) * 100, 2)}%')
    # print(f'Precision: {round(precision_score(y_test, y_pred) * 100, 2)}%')
    # print(f'Recall: {round(recall_score(y_test, y_pred) * 100, 2)}%')
    # print(f'F1: {round(f1_score(y_test, y_pred) * 100, 2)}%')
    # print(f'ROC_AUC: {round(roc_auc_score(y_test, y_pred) * 100, 2)}%')
    # print(f'Confusion Matrix:')
    # print(confusion_matrix(y_test, y_pred))

    # Get Feature Importance
    feature_importance = get_feature_importance(X, best_model)
    return feature_importance, best_model


In [71]:
ml_cols = ['entry_price', 'price_max_ratio', 'vol_max_ratio', 'if_price_max',
       'if_vol_max', 'zero_vol_mins_count', 'if_curr_higher_one',
       'if_curr_higher_open', 'if_curr_not_too_high',
       'if_curr_not_too_high_90_days', 'if_entry_after_3pm', 'entry_hour', 'if_entry_before_10am',
       'weekday']

In [72]:
xgb_predict_test_gridsearch(df_train, df_test, selected_cols=ml_cols,
                      target_encode_cols=['if_price_max',
                                        'if_vol_max', 'if_curr_higher_one', 'if_curr_higher_open',
                                        'if_curr_not_too_high', 'if_curr_not_too_high_90_days',
                                        'if_entry_after_15pm', 'if_entry_before_10am', 'entry_hour', 'weekday'], 
                      max_depth=3, learning_rate=0.2, eval_metric='logloss', 
                      proba_thresh=0.5, subsample=0.7, scale_pos_weight=10, 
                      min_child_weight=10, colsample_bytree=0.7)

Fitting 5 folds for each of 72 candidates, totalling 360 fits
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=1; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=0.7; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=1; total time=   0.4s
[CV] END colsample_bytree=0.7, learning_rate=0.1, max_depth=3, min_child_weight=1, subsample=1; total time=   0.4s
[CV] END

(                        Features     Scores
 10            if_entry_after_3pm   0.243031
 11                    entry_hour   0.145991
 12          if_entry_before_10am   0.127538
 6             if_curr_higher_one   0.106998
 0                    entry_price   0.079641
 8           if_curr_not_too_high  0.0672945
 7            if_curr_higher_open  0.0623581
 1                price_max_ratio  0.0435375
 3                   if_price_max  0.0394472
 5            zero_vol_mins_count  0.0363585
 2                  vol_max_ratio  0.0271794
 13                       weekday  0.0206266
 4                     if_vol_max          0
 9   if_curr_not_too_high_90_days          0,
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=0.7, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, gpu_id=None, grow_policy=None

In [93]:
fea_imp, y_pred = xgb_predict_test(df_train, df_test, selected_cols=ml_cols,
                      target_encode_cols=['if_price_max',
                                        'if_vol_max', 'if_curr_higher_one', 'if_curr_higher_open',
                                        'if_curr_not_too_high', 'if_curr_not_too_high_90_days',
                                        'if_entry_after_3pm', 'entry_hour', 'weekday'], 
                      max_depth=3, learning_rate=0.1, eval_metric='logloss', 
                      proba_thresh=0.6, subsample=0.7, scale_pos_weight=10, 
                      min_child_weight=10, colsample_bytree=0.7)

training data size: 6300 - target: 10.65%
testing data size: 1645 - target: 11.12%

Accuracy: 64.5%
Precision: 16.3%
Recall: 53.01%
F1: 24.94%
ROC_AUC: 59.47%
Confusion Matrix:
[[964 498]
 [ 86  97]]


In [94]:
df_ml = pd.read_csv('data/signals_backtest.csv')

df_ml['weekday'] = pd.DatetimeIndex(df_ml.date).weekday
df_ml['if_entry_before_10am'] = (df_ml['entry_hour'] == 9).astype(int)
df_ml['target'] = (df_ml['return'] == 3).astype(int)

df_ml['price_max_ratio'] = df_ml['price_max_ratio'] - 95
df_ml['vol_max_ratio'] = df_ml['vol_max_ratio'] - 70

df_ml['if_price_max'] = (df_ml['price_max_ratio'] >= 5).astype(int)
df_ml['if_vol_max'] = (df_ml['vol_max_ratio'] >= 30).astype(int)

In [95]:
t = df_ml.iloc[6300:]
t['prediction'] = y_pred.astype(int)

In [96]:
t['return'].mean()

0.0024242834836044813

In [97]:
t[t.prediction == 1]['return'].mean()

0.05820970747278771

In [98]:
t[t.prediction == 1].shape

(595, 22)

In [99]:
t[t.prediction == 0]['return'].mean()

-0.026634013171596888

In [100]:
t[t.price_max_ratio >= 5][t.vol_max_ratio >= 30][t.if_curr_higher_one == 1][t.if_curr_higher_open == 1][t.entry_hour==9]['return'].mean()

0.7027352848924182

In [101]:
t[t.prediction == 1][t.price_max_ratio >= 5][t.vol_max_ratio >= 30][t.if_curr_higher_one == 1][t.if_curr_higher_open == 1][t.entry_hour==9]['return'].mean()

0.6698694859542921

In [103]:
t[t.prediction == 0][t.price_max_ratio >= 5][t.vol_max_ratio >= 30][t.if_curr_higher_one == 1][t.if_curr_higher_open == 1][t.entry_hour==9]

,date,return,ticker,entry_time,entry_price,exit_time,exit_price,price_max_ratio,vol_max_ratio,zero_vol_mins_count,if_curr_higher_one,if_curr_higher_open,if_curr_not_too_high,if_curr_not_too_high_90_days,if_entry_after_3pm,entry_hour,weekday,if_entry_before_10am,target,if_price_max,if_vol_max,prediction
6343,2023-02-02,0.000000,PNR,09:40:00,59.620,10:00:00,59.6200,8.06,44.80,0.0,1,1,1,1,0,9,3,1,0,1,1,0
6355,2023-02-02,3.000000,RKT,09:41:00,10.100,09:53:00,10.4030,7.91,79.27,0.0,1,1,1,1,0,9,3,1,1,1,1,0
6729,2023-02-09,0.126743,CSII,09:30:00,19.725,15:59:00,19.7700,5.25,112.10,0.0,1,1,1,1,0,9,3,1,0,1,1,0
7099,2023-02-22,0.075643,INDT,09:30:00,66.100,15:59:00,66.1700,5.61,1202.02,0.0,1,1,1,1,0,9,2,1,0,1,1,0
7156,2023-02-23,-1.000000,STN,09:40:00,59.790,09:43:00,59.1921,19.52,35.88,0.0,1,1,1,1,0,9,3,1,0,1,1,0
7430,2023-03-06,NaN,UNCY,09:30:00,1.340,09:31:00,NaN,20.20,167.56,0.0,1,1,1,1,0,9,0,1,0,1,1,0
7629,2023-03-14,3.000000,AIMD,09:30:00,1.160,09:32:00,1.1948,43.98,4827.40,0.0,1,1,1,0,0,9,1,1,1,1,1,0


In [82]:
t[t.prediction == 1][t.price_max_ratio >= 5][t.vol_max_ratio >= 30][t.if_curr_higher_one == 1][t.if_curr_higher_open == 1][t.entry_hour==9].shape

(49, 22)

In [91]:
df_ml[df_ml.price_max_ratio >= 5][df_ml.vol_max_ratio >= 30][df_ml.if_curr_higher_one == 1][df_ml.if_curr_higher_open == 1][df_ml.entry_hour==9]['return'].mean()

0.43257732507878155